In [823]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
from pandas import Series,DataFrame

from numpy import linalg as linis

import random
import math
import sys

from RandomClass import Random   
from collections import Counter

%matplotlib inline

# Harmoninen värähtelijä

## Mersu usage

| command        | effect          |
| :------------- | :-------------- |
| `rndg.random()`  | return uniform distribution in [0,1)  |
| `rndg.randint(begin_number, end_number)` | return random int in [a,b) |
| `rndg.shuffle(sequence as list)` | shuffle the input sequence |
| `rndg.choice(sequence, replace=True, size=1)` | choice an element randomly in the sequence |
| `rndg.bern(p)` | generate a Bernoulli Random Variable, p: the probability of True |
| `rndg.binomial(n, p)` | generate a Binomial Random Variable, n: total times, p: probability of success | 
| `rndg.geometric(p)` | generate a Geometric Random Variable, p: probability of success |

### Making of random numbers
```
n_nums = lukujen määrä
decimals = numeroiden määrä
my_seed = randomin siemen
```

In [824]:
n_nums=10**6
decimals=3
my_seed=31415

In [825]:
rndg = Random(my_seed)   # init Mersu

#### Kokonaislukuja

In [826]:
def rPInt(decimals):                 # funktio joka tekee random positiivisen kokonaisluvun
    lower=10**(decimals-1)
    upper=10**decimals
    return rndg.randint(lower,upper)
def rInt(decimals):                 # funktio joka tekee random kokonaisluvun
    lower=-10**decimals
    upper=10**decimals
    return rndg.randint(lower,upper)

In [827]:
def rPInts():                      # funktio joka tekee listan random positiivisia kokonaislukuja
    list_rPInts = []
    for x in range(n_nums):
        list_rPInts.append(rPInt(decimals))
    return list_rPInts
def rInts():                      # funktio joka tekee listan random kokonaislukuja
    list_rInts = []
    for x in range(n_nums):
        list_rInts.append(rInt(decimals))
    return list_rInts

#### Reaalilukuja

In [828]:
def rPReals01():                 # funktio joka tekee listan random reaalilukuja [0,1)]
    list_rPReals01 = []
    for x in range(n_nums):
        list_rPReals01.append(rndg.random())
    return list_rPReals01
def rReals01():                 # funktio joka tekee listan random reaalilukuja [-1,1)]
    list_rReals01 = []
    for x in range(n_nums):
        if rndg.bern(rndg.random()):       # True todennäköisyys arvotaan joka kerta
            list_rReals01.append(rndg.random())   # True: positiivinen
        else:
            list_rReals01.append(-rndg.random())  # False: negatiivinen
    return list_rReals01

In [829]:
# for x in range(n_nums):
#    print(rndg.bern(rndg.random()))

### -1 ja 1 50:50 chance

In [830]:
def fifty_sixty():
    if rndg.bern(0.5):
        return 1
    else:
        return -1

In [831]:
def fifties_sixties():
    list_fifsixs = []
    for x in range(n_nums):
        if rndg.bern(0.5):
            list_fifsixs.append(1)
        else:
            list_fifsixs.append(-1)
    return list_fifsixs

## Leapfrog algorithm
- Forces
- new momentum based on the force and half the timestep
- new position
- next new momentum with the other half of the timestep

### Differential equations for motion
$ dx/dt = v(t) $    ------------------------------->    $ x1 = x0 + v(1/2)dt $  
$ dv/dt = 1/m * (- dU(x)/dx) $ ---------------> $ v(3/2) = v(1/2) + (- dU(x1)/dx)dt $

In [939]:
m1 = 1
m2 = 1
m_red = (m1 * m2)/(m1 + m2)    # reduced mass
m_tot = m1 + m2

k = 1                  # spring constant

In [940]:
r1 = np.array([rndg.random() * fifty_sixty(),rndg.random() * fifty_sixty()])         # location of particle1
r2 = np.array([rndg.random() * fifty_sixty(),rndg.random() * fifty_sixty()])        # location of particle1
r_cm = (m1*r1 + m2*r2)/(m_tot)  # location of CM

r_eq = 2               # distance of particle1 and particle2 at equilibrium
r12 = r2 - r1
r_eq_vec = (r2 - r1)/linis.norm((r2 - r1)) * r_eq


v1 = np.array([rndg.random() * fifty_sixty(),rndg.random() * fifty_sixty()])         # velocity of particle1
v2 = np.array([rndg.random() * fifty_sixty(),rndg.random() * fifty_sixty()])        # velocity of particle2
v12 = v2 - v1
v21 = -v12

p1 = m1 * v1             # momentum of particle1
p2 = m2 * v2             # momentum of particle2
p_cm = p1 + p2
p12 = p2 - p1

In [941]:
r12 - r_eq_vec

array([ 0.55028799, -0.08114138])

In [942]:
l12 = np.cross(r12,p12)       # angular momentum
print(l12)

-0.5949202672730569


In [943]:
r1_len = np.sqrt(r1.dot(r1)) 
r2_len = np.sqrt(r2.dot(r2)) 
r12_len = np.sqrt(r12.dot(r12))
print(r12_len)

1.4437619289973107


In [944]:
n = 0                  # index of timesteps
dt = 0.01             # size of timestep

def tstep_x():
    tstep_x = dt         # timestep for position
    return tstep_x
def tstep_v():
    tstep_v = dt - (dt*0.5)   # timestep for velocity is off-phase
    return tstep_v

n_steps = 10000       # number of timesteps

In [914]:
with open("hvibD2.xyz","w") as c:
    for i in range(n_steps+1):
        #v12_old = v12.copy()
        v12 -= (k/m_red) * (r12 - r_eq_vec) * dt
        r12 += v12 * dt
        # r12_len = np.sqrt(r12.dot(r12))
        r_cm = (m1*r1 + m2*r2)/(m_tot)  # location of CM
        
        x1 = r12[0]/2 + r_cm[0]
        y1 = r12[1]/2 + r_cm[1]
        x2 = -r12[0]/2 + r_cm[0]
        y2 = -r12[1]/2 + r_cm[1]
        
        r1 = np.array([x1,y1])
        r2 = np.array([x2,y2])
    
#        c.write("2\ntstep: {}".format(i), "\nH1 {} {} 0".format(r12[0]/2, r12[1]/2), "\nH2 {} {} 0\n".format(-r12[0]/2, -r12[1]/2))
        c.write("2\ntstep: {}\nH1 {} {} 0\nH2 {} {} 0\n".format(i, x1, y1, x2, y2))

In [945]:
with open("hvibD2_E.txt","w") as c:
    for i in range(n_steps+1):
        #energy 
        r12_len = np.sqrt(r12.dot(r12))
        v12_len = np.sqrt(v12.dot(v12))
        U = k*(r12_len - r_eq)**2
        p1 = m1 * v12_len/2 * r12/r12_len
        p2 = m2 * v12_len/2 * r12/r12_len
        K1 = np.sqrt(p1.dot(p1))**2 /(2 * m1)
        K2 = np.sqrt(p2.dot(p2))**2 /(2 * m2)
        linear_momentum = p1.dot(p2)
        l12 = np.cross(r12,p12)
        # angular_momentum = massCM * math.sqrt(ang_velocity1**2 + ang_velocity2**2) * r/2
        E_rot = l12 / (4 * r12_len * np.sqrt(r_cm.dot(r_cm)))
        E_tot = U + K1 + K2 # E_rot
        
        #v12_old = v12.copy()
        v12 -= (k/m_red) * (r12 - r_eq_vec) * dt
        r12 += v12 * dt
        
        r_cm = (m1*r1 + m2*r2)/(m_tot)  # location of CM
        
        x1 = r12[0]/2 + r_cm[0]
        y1 = r12[1]/2 + r_cm[1]
        x2 = -r12[0]/2 + r_cm[0]
        y2 = -r12[1]/2 + r_cm[1]
        
        r1 = np.array([x1,y1])
        r2 = np.array([x2,y2])
    
#        c.write("2\ntstep: {}".format(i), "\nH1 {} {} 0".format(r12[0]/2, r12[1]/2), "\nH2 {} {} 0\n".format(-r12[0]/2, -r12[1]/2))
        c.write("Step: {} \t Linear momentum = {} \t Angular momentum = {} \t U = {} \t K = {} \t E_rot = {} \t E_tot = {} \n".format(i,linear_momentum,l12,U,K1+K2,E_rot,E_tot))

In [964]:
m1 = 1
m2 = 1
m_red = (m1 * m2)/(m1 + m2)    # reduced mass
m_tot = m1 + m2

k = 1                  # spring constant


In [965]:
r1 = 1.5         # location of particle1
r2 = -1.5        # location of particle1
r_cm = (m1*r1 + m2*r2)/(m_tot)  # location of CM

r_eq = 2               # distance of particle1 and particle2 at equilibrium
r12 = np.abs(r2 - r1)

v1 = 0.0         # velocity of particle1
v2 = 0.0        # velocity of particle2
v12 = v2 - v1
v21 = -v12

p1 = m1 * v1             # momentum of particle1
p2 = m2 * v2             # momentum of particle2
p_cm = p1 + p2
p12 = p2 - p1

In [966]:
n = 0                  # index of timesteps
dt = 0.01             # size of timestep

def tstep_x(n):
    tstep_x = n*dt         # timestep for position
    return tstep_x
def tstep_v(n):
    tstep_v = (n+0.5)*dt   # timestep for velocity is off-phase
    return tstep_v

n_steps = 10000       # number of timesteps

In [949]:
with open("hvibD1.xyz","w") as c:
    for i in range(n_steps+1):
        c.write("2\ntstep: {}\nH1 {} 0 0\nH2 {} 0 0\n".format(i,r12/2, -r12/2))
        v12 -= (k/m_red) * (r12 - r_eq) * dt
        r12 += v12 * dt

In [967]:
with open ("hvibD1_fig.txt","w") as f:
    for i in range(n_steps+1):
        f.write("{},{}\n".format(i,r12-r_eq))
        v12 -= (k/m_red) * (r12 - r_eq) * dt
        r12 += v12 * dt

In [969]:
with open ("hvibD1_sin.txt","w") as f:
    for i in range(n_steps+1):
        f.write("{},{}\n".format(i,np.cos(np.sqrt(k/m_red)*dt*i)))

sin = pd.read_csv("hvibD1_sin.txt")
sindf = sin.parse()

# Laura & Tomi

In [384]:
#constants
timestep = 0.001
steps = 100000
mass1 = 1
mass2 = 1
massCM = (mass1 * mass2)/(mass1 + mass2)
spring_constant = 1
eq_distance = 1

In [385]:
#starting values
velx1 = -0.5
vely1 = -0.5
vel1 = [velx1,vely1]
velx2 = 0.5
vely2 = 0.5
vel2 = [velx2,vely2]
x1 = 0.75
y1 = 0
x2 = -0.75
y2 = 0

In [386]:
deltax = x2-x1
deltay = y2-y1
deltar = [deltax, deltay]
if (np.cross(deltar,vel1) >= 0):
    rotdir1 = -1
else:
    rotdir1 = 1
if (np.cross(deltar,vel2) >= 0):
    rotdir2 = -1
else:
    rotdir2 = 1

In [387]:
print(rotdir1, rotdir2)

-1 1


In [388]:
r = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
theta = math.atan((y2 - y1)/(x2 - x1))
if (velx1) == 0:
    alpha1 = math.pi
elif (velx2) == 0:
    alpha2 = 0
else:
#    alpha1 = math.acos(((x2 -x1)*velx1 + (y2 - y1)*vely1)/(math.sqrt((x2 - x1)**2 + (y2 - y1)**2)*math.sqrt(velx1**2 + vely1**2)))
#    alpha2 = math.acos(((x2 -x1)*velx2 + (y2 - y1)*vely2)/(math.sqrt((x2 - x1)**2 + (y2 - y1)**2)*math.sqrt(velx2**2 + vely2**2)))

    alpha1 = math.atan(vely1/velx1) + theta
    alpha2 = math.atan(vely2/velx2) + theta

In [389]:
print(alpha1, alpha2)

0.7853981633974483 0.7853981633974483


In [390]:
vel1_par = math.cos(alpha1) * math.sqrt(velx1**2 + vely1**2)
vel2_par = math.cos(alpha2) * math.sqrt(velx1**2 + vely1**2)

vel1_per = math.sin(alpha1) * math.sqrt(velx1**2 + vely1**2)
vel2_per = math.sin(alpha2) * math.sqrt(velx1**2 + vely1**2)

In [391]:
print(vel1_par, vel2_par)
print(vel1_per, vel2_per)

0.5000000000000001 0.5000000000000001
0.5 0.5


In [392]:
# combined rotational direction 1 = counter clockwise, -1 = clockwise
if (vel1_per == vel2_per) and (rotdir1 == rotdir2):   # ei pyöri
    rotdir_tot = 0
    velrot = 0
elif (vel1_per > vel2_per) and (rotdir1 == rotdir2):  # pyörii suuremman samansuuntaisen vektorin määräämään suuntaan
    rotdir_tot = rotdir1
    velrot = np.abs(vel1_per - vel2_per)  #___ rigid rotor
elif (vel1_per < vel2_per) and (rotdir1 == rotdir2):  # pyörii suuremman samansuuntaisen vektorin määräämään suuntaan
    rotdir_tot = rotdir2
    velrot = np.abs(vel2_per - vel1_per) # ___ rigid rotor
elif (rotdir1 == 1 and rotdir2 == -1):
    rotdir_tot = 1
    velrot = np.abs(vel1_per - vel2_per)  #___ rigid rotor
elif (rotdir1 == -1 and rotdir2 == 1):
    rotdir_tot = -1
    velrot = np.abs(vel2_per - vel1_per)  #___ rigid rotor
print(rotdir_tot)
print(velrot)

-1
0.0


In [393]:
ang_velocity1 = velrot
ang_velocity2 = velrot
K1 = 1/2 * mass1 * math.sqrt(velx1 ** 2 + vely1 **2)
K2 = 1/2 * mass2 * math.sqrt(velx2 ** 2 + vely2 **2)
U = (r - eq_distance) * spring_constant
E_rot = math.sqrt(ang_velocity2**2 + ang_velocity1**2) / (4 * r)

In [394]:
#defining writing step
def writestep():
    f.write("2\nstep: " + str(i) + " K1: " + str(K1) + " K2: " + str(K2)
            + " U: " + str(U) + " E_J: " + str(E_rot) + " E_tot: " + str(K1 + K2 + U + E_rot)
            + "\nH "+ str(x1) + " " + str(y1) + " 0\nH "+ str(x2) + " " + str(y2) + " 0\n")
    #f.write(str(velx1) + " vely1: " + str(vely1) + " velx2: " + str(velx2) + " r: " + str(r)
    #        + " theta: " + str(theta) + " alpha1: " + str(alpha1) + " alpha2" + str(alpha2) 
    #        + " vel1_per: " + str(vel1_per) + " vel1_par: " + str(vel1_par) 
    #        + " vel2_per: " + str(vel2_per) + " vel2_par: " + str(vel2_par) 
    #        + " ang1: " + str(ang_velocity1) + " ang2: " + str(ang_velocity2) + "\nH")

In [395]:
#opening a file
#3 ja hiili kuvaa CM:C$C$, helppo poistaa
f = open("hvib.xyz", "w")

#leapfrog (?)
i = 0
#writestep()

for i in range(steps):
    
    if i%100 == 0:
        writestep()
    #x1 += velx1 * timestep
    #y1 += vely1 * timestep
    #x2 += velx2 * timestep
    #y2 += vely2 * timestep
    #r = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    #ang_velocity1 = (ang_velocity1 + vel1_per)/ (r/2)
    #ang_velocity2 = (ang_velocity2 + vel2_per)/ (r/2)
    #theta += vel1_per * timestep
    
    theta += velrot * rotdir_tot * timestep
    # r 1 -> 2
    vel1_par -= (spring_constant/mass1) * (r - eq_distance) * timestep
    #velx1 = math.cos(alpha1)*math.cos(theta)*vel1_par + math.sin(alpha1)*math.cos(theta)*vel1_per
    #vely1 = math.cos(alpha1)*math.sin(theta)*vel1_par + math.sin(alpha1)*math.sin(theta)*vel1_per
    vel2_par += (spring_constant/mass2) * (r - eq_distance) * timestep
    #velx2 = math.cos(alpha2)*math.cos(theta)*vel2_par + math.sin(alpha2)*math.cos(theta)*vel2_per
    #vely2 = math.cos(alpha2)*math.sin(theta)*vel2_par + math.sin(alpha2)*math.sin(theta)*vel2_per
    p1 = mass1 * math.sqrt(vel1_par**2+ vel1_per**2)
    p2 = mass2 * math.sqrt(vel2_par**2+ vel2_per**2)
 
    #energy 
    U = (r - eq_distance) * spring_constant
    K1 = p1**2 /(2 *mass1)
    K2 = p2**2 /(2 *mass2)
    linear_momentum = p1 + p2
    angular_momentum = massCM * math.sqrt(ang_velocity1**2 + ang_velocity2**2) * r/2
    E_rot = math.sqrt(ang_velocity2**2 + ang_velocity1**2) / (4 * r)
    E_tot = U + K1 + K2 + E_rot
    

f.close()

In [313]:
a = np.array([1,0])  
b = np.array([0,1])  
print(np.cross(a,b))

1


# Jostain netistä

In [270]:
def SolveHarmOscLeapFrog(tfinal, h, w0, phase):
    """ Compute x(t) of the harmonic oscillator d^2x/dt^2= - w0^2 x
    """
    N = int(tfinal/h)
    h = tfinal/N
 
    x = np.zeros(N)
    v = x.copy()
 
    x[0] = math.cos(0 + phase)  # "cheating" using the analytic solution for start
    v[0] = -w0*math.sin(0 + phase)
 
## update x then v 
#    for i in range(1,N):
#        x[i] = x[i-1] + h/2 * v[i-1]
#        v[i] = v[i-1] + h * (-w0**2) * x[i]
#        x[i] = x[i] + h/2 * v[i]
# alternative form all in one go
    for i in range(1,N):
        x[i] = x[i-1] + h * v[i-1] + h*h/2* (-w0**2)*x[i-1]
        v[i] = v[i-1] + h * (-w0**2) * (x[i-1] + x[i])/2
 
    t = np.arange(N)*h
    return x,v,t

In [281]:
# Main program starts here
t = 0
tfinal = 2*2.*math.pi
w0=1
phase=1
 
hs = [0.0003,0.001,.01, .03, .1, .3]
hs.reverse()
L21 = np.zeros(len(hs))
L2euler = L21.copy()
L2eulerB = L21.copy()
L2Heun = L21.copy()
L2lf= L21.copy()
L21n = L21.copy()
L2he= L21.copy()
vL21= L21.copy()
vL2lf = L21.copy()
N = np.arange(len(hs))
i=0
print("# of points  x-SV-method      \tx-LeapFrog   \t  \t  x-Euler   \t Heun   \t \t   x-Hermite")
for i in range(len(hs)):
    h=hs[i]
# analytic solution looks like this
    xa = np.cos(w0*t+phase)
    va = -w0*np.sin(w0*t+phase)
#    L21[i]  = L2(x,xa)
#    vL21[i] = L2(v,va)
    (xn,vn,tn) = SolveHarmOscNumerov(tfinal, h, w0, phase)
    L21n[i]  = L2(xn,xa)
    (xe,ve,te) = SolveHarmOscEuler(tfinal, h, w0, phase)
    L2euler[i] = L2(xe,xa)
    (xh,vh,th) = SolveHarmOscHeun(tfinal, h, w0, phase)
    L2Heun[i] = L2(xh,xa)
    (xeb,veb,teb) = SolveHarmOscEulerBackward(tfinal, h, w0, phase)
    L2eulerB[i] = L2(xeb,xa)
    (xlf,vlf,tlf) = SolveHarmOscLeapFrog(tfinal, h, w0, phase)
    L2lf[i] = L2(xlf,xa)
    vL2lf[i]= L2(vlf,va)
    (xhe,vhe,the) = SolveHarmOscHermite(tfinal, h, w0, phase)
    L2he[i] = L2(xhe,xa)
    N[i] = int(tfinal/h)
    print(N[i], "\t:", L21[i], "\t", L2lf[i], "\t", L2euler[i], "\t", L2Heun[i], "\t", L2he[i])
 
# <codecell>
 
import matplotlib.pyplot as plt
plt.loglog(N,L2euler,'o-',N,L2eulerB,'o-',N,L2Heun,'x-',N,L21,'o-',
    N,L2lf,'x-', N, L21n, ':o', double(N),L2he,'-o',
    N,double(N)**(-2)*1e2, '-', 
    10.**(np.arange(4)+1),(10.**(np.arange(4)+1))**(-4)*2e3)
plt.legend(('Euler', 'Euler back', 'Heun', 'Verlet', 'Leap-frog', 'Numerov', 'Hermite', '$\propto $N$^{-2}$', '$\propto $N$^{-4}$'),'lower left',prop={'size':8})
plt.xlabel('Number of steps')
plt.ylabel('$L_2$ Error Norm')
plt.title('Harmonic Oscillator Solution Error')
plt.ylim((1e-14,1e1));
plt.xlim((1e1,1e5));
 
# <codecell>
 
Energy = .5 # w0*w0/2 * x[0]*x[0] +  va[0]*va[0]/2;
plt.loglog( te,fabs(0.5*w0**2*xe**2 + 0.5*ve**2- Energy),'-',
            teb,fabs(0.5*w0**2*xeb**2+0.5*veb**2- Energy),'-.',
            teb,fabs(0.5*w0**2*xh**2+0.5*vh**2- Energy),'x-',
    tlf,fabs(0.5*w0**2*xlf**2+0.5*vlf**2- Energy),'-',
     t ,fabs(0.5*w0*w0 * x*x + 0.5* v*v - Energy),':',
     t ,fabs(0.5*w0*w0 * xn*xn + 0.5* vn*vn - Energy),'-.',
     t ,fabs(0.5*w0*w0 * xhe*xhe + 0.5* vhe*vhe- Energy),'-.',
     t ,fabs(0.5*w0*w0 * x*x + 0.5* va*va- Energy),':');
plt.title('Energy vs time')
plt.xlabel('time')
plt.ylabel('Energy')
plt.legend(( 'Euler', 'Euler backward', 'Heun',
    'leap frog', 'Verlet', 'Numerov', 'Hermite', 'analytic'),'upper left',prop={'size':8});
 
# <codecell>
 
plt.plot(tlf,xlf,'-',te,xe,'o-',teb,xeb,'x-',t,x,'--',t,xa,':')
plt.legend(('leapfrog', 'Euler', 'Euler backwards', 'Verlet', 'exact'),'upper left',prop={'size':8});
 
# <headingcell level=2>
 
# The Verlet and leap frog methods are indeed identical! 
# Euler is terrible and Hermite integration is what we want if we can afford it.
 
# <codecell>

# of points  x-SV-method      	x-LeapFrog   	  	  x-Euler   	 Heun   	 	   x-Hermite


NameError: name 'SolveHarmOscNumerov' is not defined